<font size=4>**1. Capacitated Facility Location Problem**</font>

    Capacitated Facility Location Problem (CFLP) is generalization of the Simple Plant Location Problem. Each facility can produce limited quantity of products. 

<img src="cflp_model.png">

1) Bu sorudaki veri tedarik zincirini iyileştirmek isteyen bir hazır perakende satıcısından alınmıştır. Bu şirketin 5 farklı bölgedeki (USA, Germany, Japan, Brazil, and India) bölgesel talebi bilinmektedir. Ayrıca her bölgede depo işletmenin sabit maliyeti ve bölgeler arası nakliye maliyetleri de bilinmektedir. Bu şirket için talebi karşılayan en uygun kapasitedeki depoların hangi bölgelerde açılması gerektiğini bulunuz.

<font size=3>**1.1. Extended Capacitated Facility Location Problem**</font>

In [4]:
import numpy as np
import pandas as pd
from pulp import *

In [5]:
demand = pd.read_excel('capacitated_plant_location.xlsx', sheet_name='demand', index_col=0)
var_cost = pd.read_excel('capacitated_plant_location.xlsx', sheet_name='var_cost', index_col=0)
cap = pd.read_excel('capacitated_plant_location.xlsx', sheet_name='cap', index_col=0)
fix_cost = pd.read_excel('capacitated_plant_location.xlsx', sheet_name='fix_cost', index_col=0)

<font size=3>**1.1.1. Parameters**</font>

In [6]:
display(demand)
display(var_cost)
display(cap)
display(fix_cost)

,Dmd
Supply_Region,
USA,2719.6
Germany,84.1
Japan,1676.8
Brazil,145.4
India,156.4


,USA,Germany,Japan,Brazil,India
Supply_Region,,,,,
USA,6,13,20,12,17
Germany,13,6,14,14,13
Japan,20,14,3,21,9
Brazil,12,14,21,8,21
India,22,13,10,23,8


,Low_Cap,High_Cap
Supply_Region,,
USA,500,1500
Germany,500,1500
Japan,500,1500
Brazil,500,1500
India,500,1500


,Low_Cap,High_Cap
Supply_Region,,
USA,6500,9500
Germany,4980,7270
Japan,6230,9100
Brazil,3230,4730
India,2110,3080


<font size=3>**1.1.2. Pandas iloc & loc**</font>

In [5]:
# Pandas Loc
cap.iloc[1,1]

1500

In [6]:
cap.loc["USA", "Low_Cap"]

500

<font size=3>**1.1.3. Model**</font>

In [39]:
cap.loc['USA', "Low_Cap"]

500

In [51]:
# Initialize, and Define Decision Vars.
model = LpProblem("Extended Capacitated Plant Location Model", LpMinimize)

#Ranges
wh = ['USA', 'Germany', 'Japan', 'Brazil', 'India']
store = ['USA', 'Germany', 'Japan', 'Brazil', 'India']
size = ['Low_Cap','High_Cap']

#number of goods shipped from wh i to store j
x = LpVariable.dicts("goods_", [(i,j) for i in wh for j in store], lowBound=0, cat='Continuous')
y = LpVariable.dicts("wh_", [(i,s) for i in wh for s in size], cat='Binary')

#objective function
model += lpSum([var_cost.loc[i,j] * x[(i,j)] for i in wh for j in store]) + lpSum([fix_cost.loc[i,s] * y[(i,s)] for i in wh for s in size])

#constraints
#demand constraint
for j in store:
    model += lpSum([x[(i,j)] for i in wh]) == demand.loc[j]

#capacity constraint
for i in wh:
    model += lpSum([x[(i,j)] for j in store]) - lpSum([cap.loc[i,s] * y[(i,s)] for s in size])  <= 0

In [53]:
#default solver
model.solve()

1

In [57]:
# Print Objective Fun. Value
print("Total Cost = ", value(model.objective))

Total Cost =  58850.899999999994


In [58]:
# Print variables
for v in model.variables():
    print(v.name, "=", v.varValue)

goods__('Brazil',_'Brazil') = 145.4
goods__('Brazil',_'Germany') = 0.0
goods__('Brazil',_'India') = 0.0
goods__('Brazil',_'Japan') = 0.0
goods__('Brazil',_'USA') = 1219.6
goods__('Germany',_'Brazil') = 0.0
goods__('Germany',_'Germany') = 0.0
goods__('Germany',_'India') = 0.0
goods__('Germany',_'Japan') = 0.0
goods__('Germany',_'USA') = 0.0
goods__('India',_'Brazil') = 0.0
goods__('India',_'Germany') = 84.1
goods__('India',_'India') = 156.4
goods__('India',_'Japan') = 176.8
goods__('India',_'USA') = 0.0
goods__('Japan',_'Brazil') = 0.0
goods__('Japan',_'Germany') = 0.0
goods__('Japan',_'India') = 0.0
goods__('Japan',_'Japan') = 1500.0
goods__('Japan',_'USA') = 0.0
goods__('USA',_'Brazil') = 0.0
goods__('USA',_'Germany') = 0.0
goods__('USA',_'India') = 0.0
goods__('USA',_'Japan') = 0.0
goods__('USA',_'USA') = 1500.0
wh__('Brazil',_'High_Cap') = 1.0
wh__('Brazil',_'Low_Cap') = 0.0
wh__('Germany',_'High_Cap') = 0.0
wh__('Germany',_'Low_Cap') = 0.0
wh__('India',_'High_Cap') = 0.0
wh__('Indi

In [7]:
#default solver
model.solve()

#with cplex
model.solve(solver=CPLEX())

1

<font size=4>**2. CPLEX Tools**</font>

In [8]:
#status of CPLEX
print(CPLEX().available())
print(CPLEX().defaultPath())

C:\Program Files\IBM\ILOG\CPLEX_Studio1263\cplex\bin\x64_win64\cplex.exe
cplex.exe


In [9]:
#Solving with CPLEX
display(CPLEX().actualSolve(model))
display(model.solve(solver=CPLEX()))

1

1

<font size=4>**3. Sensitivity Analysis**</font>

2) Bu soruda bir beyaz eşya üreticisinin 4 aylık üretim planını oluşturmanız isteniyor. Aylık üretim kapasitesi, talep, üretim gideri, envanter gideri ve sabit gider d dataframe'inde veriliyor. Ayrıca, başlangıç envanteri olarak depoda 100 ürün var. 

En düşük masrafı sağlamak için, aylık üretim adetlerini ve her ay sonunda envanterde kaç ürün kalması gerektiğini bulunuz.

**minimize**
20 * x_1 + 24 * x_2 + 30 * x_3 + 18 * x_4 + 5 * s_1 + 5 * s_2 + 5 * s_3 + 5 * s_4 + 80000 * y_1 + 80000 * y_2 + 80000 * y_3 + 80000 * y_4

**subject to:**<br>
C1: x_1 - s_1 = 880<br>
C2: - 1200 y_1 + x_1 <= 0<br>
C3: x_2 + s_1 - s_2 = 870<br>
C4: - 1000 y_2 + x_2 <= 0<br>
C5: x_3 + s_2 - s_3 = 850<br>
C6: - 900 y_3 + x_3 <= 0<br>
C7: x_4 + s_3 - s_4 = 970<br>
C8: - 800 y_4 + x_4 <= 0<br>

x, s Integer<br>
y Binary

In [59]:
d = pd.DataFrame(np.array([[1, 80000, 20, 5, 1200, 980], [2, 80000, 24, 5, 1000, 870], [3, 80000, 30, 5, 900, 850], [4, 80000, 18, 5, 800, 970]]), 
                 columns=['period', 'fixed_setup', 'unit_prod', 'unit_inv', 'prod_cap', 'demand'])
d.reset_index(drop=True)
d.set_index('period', inplace=True)
time = [1, 2, 3, 4]

In [60]:
display(d)

,fixed_setup,unit_prod,unit_inv,prod_cap,demand
period,,,,,
1,80000,20,5,1200,980
2,80000,24,5,1000,870
3,80000,30,5,900,850
4,80000,18,5,800,970


In [76]:
model1 = LpProblem("Production Planning", LpMinimize)

x = LpVariable.dicts("amount_produced_", [t for t in time], lowBound=0, cat="Integer")
y = LpVariable.dicts("plant_on_", [t for t in time], cat="Binary")
s = LpVariable.dicts("inventory_end_of_month_", [0, 1, 2, 3, 4], lowBound=0, cat="Integer")

model1 += lpSum([d.loc[t,"unit_prod"] * x[t] + d.loc[t,"unit_inv"] * s[t] + d.loc[t,"fixed_setup"] * y[t] for t in time])

s[0] = 100

for t in time:
    model1 += (s[t-1] +  x[t] - s[t] == d.loc[t,"demand"])
    model1 += (x[t] <=  d.loc[t,"prod_cap"] * y[t])

In [84]:
model1.solve()

1

<font size=3> **Model Status** </font>

In [85]:
# Print status
print(LpStatus[model1.status])

Optimal


<font size=3> **Objective Fun. Value** </font>

In [86]:
# Print Objective Fun. Value
print("Total Cost = ", value(model1.objective))

Total Cost =  404200.0


<font size=3> **Variable Values** </font>

In [87]:
# Print variables
for v in model1.variables():
    print(v.name, "=", v.varValue)

amount_produced__1 = 1200.0
amount_produced__2 = 1000.0
amount_produced__3 = 570.0
amount_produced__4 = 800.0
inventory_end_of_month__1 = 320.0
inventory_end_of_month__2 = 450.0
inventory_end_of_month__3 = 170.0
inventory_end_of_month__4 = 0.0
plant_on__1 = 1.0
plant_on__2 = 1.0
plant_on__3 = 1.0
plant_on__4 = 1.0


<font size=3> **Variable Values to pandas dataframe** </font>

In [91]:
# Variables to pandas DF
o = [{'prod':x[t].varValue, 'stock':s[t].varValue} for t in time]
display(pd.DataFrame(o, index=time))

,prod,stock
1,1200.0,320.0
2,1000.0,450.0
3,570.0,170.0
4,800.0,0.0


<font size=3> **Slack Variables and Shadow Prices**</font>

In [93]:
# Print the Constraint Slacks and Shadow Prices
o = [{'name':name, 'slack':c.slack, 'shadow':c.pi} for name, c in model1.constraints.items()]
display(pd.DataFrame(o))

,name,shadow,slack
0,_C1,0.0,-0.0
1,_C2,0.0,-0.0
2,_C3,0.0,-0.0
3,_C4,0.0,-0.0
4,_C5,0.0,-0.0
5,_C6,0.0,330.0
6,_C7,0.0,-0.0
7,_C8,0.0,-0.0


In [94]:
model

Production Planning:
MINIMIZE
80000*plant_on__1 + 80000*plant_on__2 + 80000*plant_on__3 + 80000*plant_on__4 + 20*prod_in_1 + 24*prod_in_2 + 30*prod_in_3 + 18*prod_in_4 + 5*stock_in_1 + 5*stock_in_2 + 5*stock_in_3 + 5*stock_in_4 + 0
SUBJECT TO
_C1: prod_in_1 - stock_in_1 = 880

_C2: - 1200 plant_on__1 + prod_in_1 <= 0

_C3: prod_in_2 + stock_in_1 - stock_in_2 = 870

_C4: - 1000 plant_on__2 + prod_in_2 <= 0

_C5: prod_in_3 + stock_in_2 - stock_in_3 = 850

_C6: - 900 plant_on__3 + prod_in_3 <= 0

_C7: prod_in_4 + stock_in_3 - stock_in_4 = 970

_C8: - 800 plant_on__4 + prod_in_4 <= 0

VARIABLES
0 <= plant_on__1 <= 1 Integer
0 <= plant_on__2 <= 1 Integer
0 <= plant_on__3 <= 1 Integer
0 <= plant_on__4 <= 1 Integer
0 <= prod_in_1 Integer
0 <= prod_in_2 Integer
0 <= prod_in_3 Integer
0 <= prod_in_4 Integer
0 <= stock_in_1 Integer
0 <= stock_in_2 Integer
0 <= stock_in_3 Integer
0 <= stock_in_4 Integer